# analyze_other_metrics
Compute membrane thickness, compressibility, and torque density for result_LM_1 trajectories

In [ ]:
import csv
import glob
import pandas as pd
import numpy as np
import MDAnalysis as mda

In [ ]:
def membrane_thickness(universe, selection='segid MEMB* and name P', time_range=(1000, 1500)):
    """
    Compute membrane thickness using phosphate (P) atoms.

    Parameters
    ----------
    universe : MDAnalysis.Universe
        Loaded MDAnalysis Universe with trajectory.
    selection : str
        Atom selection string for phosphate atoms (default: 'segid MEMB* and name P').
    time_range : tuple of (float, float)
        Only include frames with ts.time within [time_range[0], time_range[1]].

    Returns
    -------
    times : np.ndarray
        Array of frame times within the specified range.
    thicknesses : np.ndarray
        Array of thickness values (z_max - z_min) per frame.
    """
    atoms = universe.select_atoms(selection)
    times = []
    thicknesses = []

    for ts in universe.trajectory:
        t = ts.time
        if t < time_range[0] or t > time_range[1]:
            continue
        zs = atoms.positions[:, 2]
        thicknesses.append(np.max(zs) - np.min(zs))
        times.append(t)

    return np.array(times), np.array(thicknesses)

In [ ]:
def compute_compressibility_modulus(
    universe,
    time_range=(0.0, 1500.0),
    nblock=3,
    temperature=310.15
):
    """
    Compute the area compressibility modulus (K_A) over a trajectory,
    based on the manuscript formula:
        K_A = (k_B * T * ⟨A⟩) / Var(A)
    but averaged over 'nblock' time blocks.

    Parameters
    ----------
    universe : MDAnalysis.Universe
        Loaded MD trajectory with .trajectory and .dimensions attributes.
    time_range : tuple (t_min, t_max)
        Only include frames with ts.time in [t_min, t_max].
    nblock : int
        Number of equal‐sized blocks to split the data into before averaging.
    temperature : float
        Simulation temperature in Kelvin.

    Returns
    -------
    KA_mean : float
        Mean compressibility modulus across blocks.
    KA_sd   : float
        Standard deviation of the block compressibility values.
    """
    kB = 1.380649e-23  # Boltzmann constant, J/K

    # 1) Collect projected bilayer areas within time range
    areas = []
    for ts in universe.trajectory:
        t = ts.time
        if t < time_range[0] or t > time_range[1]:
            continue
        Lx, Ly = ts.dimensions[:2]
        areas.append(Lx * Ly)
    areas = np.array(areas)
    
    # 2) Split into nblock blocks (last frames included in final block)
    ndata = len(areas)
    if ndata < nblock:
        raise ValueError(f"Not enough frames ({ndata}) for {nblock} blocks.")
    block_size = ndata // nblock
    offset = ndata % nblock

    KA_blocks = []
    for i in range(nblock):
        start = i * block_size + offset
        end   = start + block_size
        block = areas[start:end]
        A_mean = block.mean()
        varA   = block.var(ddof=0)
        KA = (kB * temperature * A_mean) / varA
        KA_blocks.append(KA)

    KA_blocks = np.array(KA_blocks)
    return KA_blocks.mean(), KA_blocks.std()

In [ ]:
def torque_density_from_log(log_file: str) -> (float, float):
    """
    Compute the torque density τ from a pressure profile file according to the manuscript:
    
        τ = ∫ z [P_N(z) – P_T(z)] dz
    
    where:
      • P_N(z) = P_zz(z) is the normal pressure component,
      • P_T(z) = (P_xx(z) + P_yy(z)) / 2 is the tangential pressure,
      • z is the position along the membrane normal.
    
    Expects an XVG file (or whitespace-delimited text) with four columns:
        z  P_xx  P_yy  P_zz
    
    Parameters
    ----------
    profile_file : str
        Path to the pressure profile file.
    
    Returns
    -------
    float
        Torque density (in units of pressure·length, e.g., bar·Å) integrated over z.
    """
    tau_list = []
    with open(log_file) as f:
        for line in f:
            if not line.startswith('PRESSUREPROFILE:'):
                continue
            vals   = np.array(line.split()[2:], float)
            groups = vals.reshape(-1, 3)               # columns: Pxx, Pyy, Pzz
            z      = np.arange(groups.shape[0])       # bin centers as indices
            P_tang = 0.5 * (groups[:,0] + groups[:,1]) # lateral pressure
            diff   = groups[:,2] - P_tang              # normal – tangential
            tau    = np.trapz(z * diff, x=z)
            tau_list.append(tau)
    arr = np.array(tau_list)
    return float(arr.mean()) if arr.size else 0.0, float(arr.std(ddof=0)) if arr.size else 0.0

In [ ]:
# 1) Define input paths
psf        = 'data/structure/result_LM_1.psf'
dict_dir   = 'data/misc'  # contains pressure_profiles_protonated.log & pressure_profiles_neutral.log
traj_specs = [
    ('protonated', 'data/MD_trajectory_protonated/result_LM_1/*.dcd'),
    ('neutral',    'data/MD_trajectory_neutral/result_LM_1/*.dcd')
]

results = []
for label, traj_pattern in traj_specs:
    # 2) Load trajectory
    u = mda.Universe(psf, *glob.glob(traj_pattern))
    
    # 3) Membrane thickness
    _, thicknesses = membrane_thickness(
        u,
        selection='segid MEMB* and name P',
        time_range=(1000, 1500)
    )
    
    # 4) Compressibility
    comp, comp_sd = compute_compressibility_modulus(
        u,
        time_range=(1000, 1500),
        nblock=3,
        temperature=310.15
    )
    
    # 5) Torque density from merged log
    log_file    = f"{dict_dir}/pressure_profiles_{label}_sample.log"
    torque, torque_sd = torque_density_from_log(log_file)
    
    # 6) Store results for this label
    results.append({
        'Formulation':          'result_LM_1',
        'Label':                label,
        'MembraneThickness':    float(th_vals.mean()),
        'MembraneThickness_sd': float(th_vals.std()),
        'Compressibility':      comp,
        'Compressibility_sd':   comp_sd,
        'TorqueDensity':        torque,
        'TorqueDensity_sd':     torque_sd
    })
    
# 7) Convert to DataFrame and write separate CSVs
df = pd.DataFrame(results)
df[df.Label=='protonated'].drop(columns='Label')\
  .to_csv('results/other_metrics_results_protonated.csv', index=False)
df[df.Label=='neutral'].drop(columns='Label')\
  .to_csv('results/other_metrics_results_neutral.csv',       index=False)